In [2]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import json
import pandas as pd

from tqdm.notebook import tqdm

In [3]:
data = pd.read_json("../data/nli/multinli_1.0_train.jsonl", lines=True)
data.head()

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],government,neutral,31193n,31193,Conceptually cream skimming has two basic dime...,( ( Conceptually ( cream skimming ) ) ( ( has ...,(ROOT (S (NP (JJ Conceptually) (NN cream) (NN ...,Product and geography are what make cream skim...,( ( ( Product and ) geography ) ( ( are ( what...,(ROOT (S (NP (NN Product) (CC and) (NN geograp...
1,[entailment],telephone,entailment,101457e,101457,you know during the season and i guess at at y...,( you ( ( know ( during ( ( ( the season ) and...,(ROOT (S (NP (PRP you)) (VP (VBP know) (PP (IN...,You lose the things to the following level if ...,( You ( ( ( ( lose ( the things ) ) ( to ( the...,(ROOT (S (NP (PRP You)) (VP (VBP lose) (NP (DT...
2,[entailment],fiction,entailment,134793e,134793,One of our number will carry out your instruct...,( ( One ( of ( our number ) ) ) ( ( will ( ( (...,(ROOT (S (NP (NP (CD One)) (PP (IN of) (NP (PR...,A member of my team will execute your orders w...,( ( ( A member ) ( of ( my team ) ) ) ( ( will...,(ROOT (S (NP (NP (DT A) (NN member)) (PP (IN o...
3,[entailment],fiction,entailment,37397e,37397,How do you know? All this is their information...,( ( How ( ( ( do you ) know ) ? ) ) ( ( All th...,(ROOT (S (SBARQ (WHADVP (WRB How)) (SQ (VBP do...,This information belongs to them.,( ( This information ) ( ( belongs ( to them )...,(ROOT (S (NP (DT This) (NN information)) (VP (...
4,[neutral],telephone,neutral,50563n,50563,yeah i tell you what though if you go price so...,( yeah ( i ( ( tell you ) ( what ( ( though ( ...,(ROOT (S (VP (VB yeah) (S (NP (FW i)) (VP (VB ...,The tennis shoes have a range of prices.,( ( The ( tennis shoes ) ) ( ( have ( ( a rang...,(ROOT (S (NP (DT The) (NN tennis) (NNS shoes))...


In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

# bart-large-mnli

In [5]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")
model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli", num_labels=3)
model = model.to(device)

In [ ]:
logits = []
probs = []
for idx, row in tqdm(data.iterrows()):
    with torch.no_grad():
        inputs = tokenizer.encode_plus(row["sentence1"], row["sentence2"], 
                                       add_special_tokens=True,
                                       return_tensors="pt").to(device)
        outputs = model(**inputs)
    probs.append(F.softmax(outputs.logits)[0].to("cpu"))
    logits.append(outputs.logits[0].tolist())
    del inputs, outputs
    torch.cuda.empty_cache()

0it [00:00, ?it/s]

/root/anaconda3/envs/mnli/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


In [ ]:
data["logits"] = logits
for i in range(len(probs)):
    probs[i] = probs[i].tolist()
data["probs"] = probs

In [ ]:
# prediction indexes for each model may vary
for i in tqdm(range(len(data))):
    logits = []
    probs = []
    logits.append(data[i]['logits'][0][2])
    logits.append(data[i]['logits'][0][0])
    logits.append(data[i]['logits'][0][1])
    probs.append(data[i]['probs'][0][2])
    probs.append(data[i]['probs'][0][0])
    probs.append(data[i]['probs'][0][1])
    data[i]['logits'] = logits
    data[i]['probs'] = probs

In [ ]:
data.head()

In [ ]:
json_file = data.to_json(orient='records', lines=True)
with open("bart-large-mnli-predicted.jsonl", 'w') as f:
    f.write(json_file)

# roberta-large-mnli

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli", num_labels=3)
model = model.to(device)

In [ ]:
logits = []
probs = []
for idx, row in tqdm(data.iterrows()):
    with torch.no_grad():
        inputs = tokenizer.encode_plus(row["sentence1"], row["sentence2"], 
                                       add_special_tokens=True,
                                       return_tensors="pt").to(device)
        outputs = model(**inputs)
    probs.append(F.softmax(outputs.logits)[0].to("cpu"))
    logits.append(outputs.logits[0].tolist())
    del inputs, outputs
    torch.cuda.empty_cache()

In [ ]:
data["logits"] = logits
for i in range(len(probs)):
    probs[i] = probs[i].tolist()
data["probs"] = probs

In [ ]:
# prediction indexes for each model may vary
for i in tqdm(range(len(data))):
    logits = []
    probs = []
    logits.append(data[i]['logits'][0][2])
    logits.append(data[i]['logits'][0][0])
    logits.append(data[i]['logits'][0][1])
    probs.append(data[i]['probs'][0][2])
    probs.append(data[i]['probs'][0][0])
    probs.append(data[i]['probs'][0][1])
    data[i]['logits'] = logits
    data[i]['probs'] = probs

In [ ]:
data.head()

In [ ]:
json_file = data.to_json(orient='records', lines=True)
with open("roberta-large-mnli-predicted.jsonl", 'w') as f:
    f.write(json_file)

# bert-base-mnli

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ishan/bert-base-uncased-mnli")
model = AutoModelForSequenceClassification.from_pretrained("ishan/bert-base-uncased-mnli", num_labels=3)
model = model.to(device)

In [ ]:
logits = []
probs = []
for idx, row in tqdm(data.iterrows()):
    with torch.no_grad():
        inputs = tokenizer.encode_plus(row["sentence1"], row["sentence2"], 
                                       add_special_tokens=True,
                                       return_tensors="pt").to(device)
        outputs = model(**inputs)
    probs.append(F.softmax(outputs.logits)[0].to("cpu"))
    logits.append(outputs.logits[0].tolist())
    del inputs, outputs
    torch.cuda.empty_cache()

In [ ]:
data["logits"] = logits
for i in range(len(probs)):
    probs[i] = probs[i].tolist()
data["probs"] = probs

In [ ]:
# prediction indexes for each model may vary
for i in tqdm(range(len(data))):
    logits = []
    probs = []
    logits.append(data[i]['logits'][1])
    logits.append(data[i]['logits'][0])
    logits.append(data[i]['logits'][2])
    probs.append(data[i]['probs'][1])
    probs.append(data[i]['probs'][0])
    probs.append(data[i]['probs'][2])
    data[i]['logits'] = logits
    data[i]['probs'] = probs

In [ ]:
data.head()

In [ ]:
json_file = data.to_json(orient='records', lines=True)
with open("bert-base-mnli-predicted.jsonl", 'w') as f:
    f.write(json_file)